<a href="https://colab.research.google.com/github/supertime1/BP_PPG/blob/master/Model_Analysis_and_Visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1.Introduction



This notebook analyze the BP-DL model in the following aspects:


*   Accuracy
*   Computational Cost
*   Model Visualization










#2. Setup Environment

In [ ]:
from IPython.display import display
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext tensorboard
import numpy as np
import os
import shutil
import glob
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.models import load_model 
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.layers import Conv1D, BatchNormalization, Input, Add, Activation,\
MaxPooling1D,Dropout,Flatten,TimeDistributed,Bidirectional,Dense,LSTM, ZeroPadding1D, \
AveragePooling1D,GlobalMaxPooling1D, Concatenate, Permute, Dot, Multiply, RepeatVector,\
Lambda, Average
from tensorflow.keras.initializers import glorot_uniform
import tensorflow_datasets as tfds
import multiprocessing
from datetime import datetime
import sklearn.metrics
import itertools
import io
import pickle
print(tf.__version__)

2.2.0


# 3.Model Performance

In [ ]:
os.chdir(r"C:\Users\57lzhang.US04WW4008\Desktop\Blood pressure\BP data\models\125Hz 10mmHg\ResNet-18+Attention")
model = tf.keras.models.load_model('CCCC-ResNet-18+Attention+10ts-93-0.9912.hdf5')

## 3.1 MAE

In [ ]:
with open('C:/Users/57lzhang.US04WW4008/Desktop/Blood pressure/BP data/test/test_data', "rb") as fp:
  test_data = pickle.load(fp)
test_data = np.asarray(test_data)

In [ ]:
with open("C:/Users/57lzhang.US04WW4008/Desktop/Blood pressure/BP data/test/test_labels", "rb") as fp:
  test_labels = pickle.load(fp)
test_labels = np.asarray(test_labels)

In [ ]:
#calculate number of elements in training for later use in shuffle and model.fit
number_of_test_element = 0
for i in range(len(test_label_fn)):
  label = pickle.load(open(test_label_fn[i],'rb'))
  number_of_test_element += len(label)
print("There are in total", number_of_test_element, "in test dataset")

There are in total 13197 in test dataset


In [ ]:
n_a=32 #Number of pre-LSTM states
n_s=16 #Number of post-LSTM states
t_s0 = np.zeros((number_of_test_element, n_s))
t_c0 = np.zeros((number_of_test_element, n_s))

In [ ]:
bp_estimate=model.predict([test_data,t_s0,t_c0])

In [ ]:
from sklearn.metrics import mean_absolute_error
sys_mae = mean_absolute_error(test_labels[:,0], bp_estimate[:,0])
dia_mae = mean_absolute_error(test_labels[:,1], bp_estimate[:,1])
print("Systolic MAE in test dataset:", round(sys_mae,1))
print("Diastolic MAE in test dataset:", round(dia_mae,1))

Systolic MAE in test dataset: 5.9
Diastolic MAE in test dataset: 3.5


##3.2 Mean Difference

In [ ]:
sys_diff_mean = np.mean(bp_estimate[:,0] - test_labels[:,0])
sys_diff_std = np.std(bp_estimate[:,0] - test_labels[:,0])
print("TEST DATASET: Mean value of Systolic BP differece between predicted value and ground truth:", round(sys_diff_mean,1))
print("TEST DATASET: Standard deviation of Systolic BP difference between predicted value and ground truth:", round(sys_diff_std,1))

TEST DATASET: Mean value of Systolic BP differece between predicted value and ground truth: 0.1
TEST DATASET: Standard deviation of Systolic BP difference between predicted value and ground truth: 9.7


In [ ]:
dia_diff_mean = np.mean(bp_estimate[:,1] - test_labels[:,1])
dia_diff_std = np.std(bp_estimate[:,1] - test_labels[:,1])
print("TEST DATASET: Mean value of Diastolic BP differece between predicted value and ground truth:", round(dia_diff_mean,1))
print("TEST DATASET: Standard deviation of Diastolic BP difference between predicted value and ground truth:", round(dia_diff_std,1))

TEST DATASET: Mean value of Diastolic BP differece between predicted value and ground truth: -0.0
TEST DATASET: Standard deviation of Diastolic BP difference between predicted value and ground truth: 5.7


##3.3 Hypertension Assessment

In [ ]:
#Confusion Matrix

#4.Model Computational Analysis

##4.1 FLOPS

In [ ]:
def get_flops(model_h5_path):
    session = tf.compat.v1.Session()
    graph = tf.compat.v1.get_default_graph()
        

    with graph.as_default():
        with session.as_default():
            model = tf.keras.models.load_model(model_h5_path)

            run_meta = tf.compat.v1.RunMetadata()
            opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
        
            # We use the Keras session graph in the call to the profiler.
            flops = tf.compat.v1.profiler.profile(graph=graph,
                                                  run_meta=run_meta, cmd='op', options=opts)
        
            return flops.total_float_ops

In [ ]:
FLOPS = get_flops('CCCC-Resnet18+LSTM+10ts-72-8.5513.hdf5')
FLOPS

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


11369834

##4.2 Layer FLOPS

#5.Model Visualization

#5.1 Salient regions (t-SNE)

##5.2 Low-level feature maps

##5.3 High-level feature space